# Books Recommandation System

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport

C:\Users\yskon\AppData\Local\Temp\ipykernel_13528\2397831427.py:5: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [2]:
org_df_books=pd.read_csv("./Books.csv")
org_df_ratings=pd.read_csv("./Ratings.csv")
org_df_users=pd.read_csv("./Users.csv")
#Making a copy of data
df_books = org_df_books.copy()
df_ratings = org_df_ratings.copy()
df_users = org_df_users.copy()

C:\Users\yskon\AppData\Local\Temp\ipykernel_13528\2248442972.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  org_df_books=pd.read_csv("./Books.csv")


In [3]:
df_books.head()
df_ratings.head()
df_users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [4]:


#Shape of the data
print('df_books shape:',df_books.shape)
print('df_ratings shape:',df_ratings.shape)
print('df_users shape:',df_users.shape)



df_books shape: (271360, 8)
df_ratings shape: (1149780, 3)
df_users shape: (278858, 3)


In [5]:
#info of the df_books
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


There are missing values

In [6]:
#info of the df_ratings
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [7]:


#info of the df_users
df_users.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB



Conclusion:

    Age has missing values.
    Age datatype is float.



## Preprocessing

In [8]:
def Preprocessing(df_books,df_ratings,df_users):
# Convert http to https
    df_books['Image-URL-S']=df_books['Image-URL-S'].str.replace(r'^http\b', 'https', regex=True)
    df_books['Image-URL-M']=df_books['Image-URL-M'].str.replace(r'^http\b', 'https', regex=True)
    df_books['Image-URL-L']=df_books['Image-URL-L'].str.replace(r'^http\b', 'https', regex=True)


### Merging all dataframe in once
#joining df_ratings and df_books
    df_ratings_books = df_ratings.merge(df_books, on='ISBN')
#joining df_ratings_books and df_users
    df_complete = df_ratings_books.merge(df_users, on='User-ID')
    return df_complete

In [9]:
df_complete = Preprocessing(df_books,df_ratings,df_users)

In [16]:
def Unique_value(df_complete):
    for feature in df_complete:
        uni=df_complete[feature].nunique()
        print(f"Unique in {feature}:",uni)
    

In [17]:
Unique_value(df_complete)

Unique in User-ID: 92106
Unique in ISBN: 270151
Unique in Book-Rating: 11
Unique in Book-Title: 241071
Unique in Book-Author: 101588
Unique in Year-Of-Publication: 202
Unique in Publisher: 16729
Unique in Image-URL-S: 269842
Unique in Image-URL-M: 269842
Unique in Image-URL-L: 269839
Unique in Location: 22480
Unique in Age: 141


In [18]:
# EDA
report = ProfileReport(df_complete)
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]


## Popularity Based Recommender System

It is a type of recommendation system which works on the principle of popularity and or anything which is in trend. These systems check about the product or movie which are in trend or are most popular among the users and directly recommend those.


In [21]:
def Popularity_Recommandation(df_complete,df_books):
    # number of ratings in each book
    num_rating_df = df_complete.groupby('Book-Title').count()['Book-Rating'].reset_index()
    num_rating_df.rename(columns={'Book-Rating':'Num_ratings'},inplace=True)
    # average  ratings in each book
    avg_rating_df = df_complete.groupby('Book-Title').mean()['Book-Rating'].reset_index()
    avg_rating_df.rename(columns={'Book-Rating':'Avg_ratings'},inplace=True)
    # joining num_rating_df and avg_rating_df
    popular_df = num_rating_df.merge(avg_rating_df, on='Book-Title')
    popular_df = popular_df[popular_df['Num_ratings']>=250].sort_values('Avg_ratings', ascending=False) 
    # joining popular_df and df_books
    popular_df = popular_df.merge(df_books, on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','Num_ratings','Avg_ratings']]
    return popular_df

In [23]:
popular_df = Popularity_Recommandation(df_complete,df_books)
popular_df

C:\Users\yskon\AppData\Local\Temp\ipykernel_13528\378036772.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_rating_df = df_complete.groupby('Book-Title').mean()['Book-Rating'].reset_index()


,Book-Title,Book-Author,Image-URL-M,Num_ratings,Avg_ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,https://images.amazon.com/images/P/0439136350....,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,https://images.amazon.com/images/P/0439139597....,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,https://images.amazon.com/images/P/0590353403....,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,https://images.amazon.com/images/P/043935806X....,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,https://images.amazon.com/images/P/0439064872....,556,5.183453
...,...,...,...,...,...
716,Vinegar Hill (Oprah's Book Club (Paperback)),A. Manette Ansay,https://images.amazon.com/images/P/0380730138....,265,2.245283
717,Whispers,BELVA PLAIN,https://images.amazon.com/images/P/0440216745....,286,2.199301
727,Presumed Innocent,Scott Turow,https://images.amazon.com/images/P/0446359866....,294,2.139456
733,Isle of Dogs,Patricia Cornwell,https://images.amazon.com/images/P/0425182908....,288,2.000000


## Collaborative Filtering Based Recommender System

In Collaborative Filtering, we tend to find similar users and recommend what similar users like. In this type of recommendation system, we don’t use the features of the item to recommend it, rather we classify the users into the clusters of similar types, and recommend each user according to the preference of its cluster.

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
def Preprocessed_Collaborative(df_complete):
    # Users who are giving more than 200 Book-Rating
    temp_x = df_complete.groupby('User-ID').count()['Book-Rating'] > 200
    filtered_user_id = temp_x[temp_x].index
    

    # All data of users who are giving more than 200 Book-Rating
    filtered_ratings = df_complete[df_complete['User-ID'].isin(filtered_user_id)]
    # All data of books who have more than 50 Book-Rating
    temp_y = filtered_ratings.groupby('Book-Title').count()['Book-Rating']>=50
    famous_books = temp_y[temp_y].index
    

    # All data of famous books
    final_ratings = filtered_ratings[filtered_ratings['Book-Title'].isin(famous_books)]
    

    # All the users and their rating on each famous books
    pt = final_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')
    pt.fillna(0, inplace=True)
    return pt





In [32]:
pt =Preprocessed_Collaborative(df_complete)

In [33]:
similarity_scores = cosine_similarity(pt)
similarity_scores

array([[1.        , 0.10255025, 0.01220856, ..., 0.12110367, 0.07347567,
        0.04316046],
       [0.10255025, 1.        , 0.2364573 , ..., 0.07446129, 0.16773875,
        0.14263397],
       [0.01220856, 0.2364573 , 1.        , ..., 0.04558758, 0.04938579,
        0.10796119],
       ...,
       [0.12110367, 0.07446129, 0.04558758, ..., 1.        , 0.07085128,
        0.0196177 ],
       [0.07347567, 0.16773875, 0.04938579, ..., 0.07085128, 1.        ,
        0.10602962],
       [0.04316046, 0.14263397, 0.10796119, ..., 0.0196177 , 0.10602962,
        1.        ]])

In [34]:
def Collaborative_Recommandation(book_name):
    # index fetch
    index = np.where(pt.index == book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])), key=lambda x: x[1], reverse = True)[1:6]
    
    data = []
    for i in similar_items:
        item = []
        #print(pt.index[i[0]])
        temp_df = df_books[df_books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title']))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author']))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M']))
        data.extend(item)
    return data

In [35]:
Collaborative_Recommandation('Nights in Rodanthe')

['The Mulberry Tree',
 'Jude Deveraux',
 'https://images.amazon.com/images/P/0743437640.01.MZZZZZZZ.jpg',
 'Dark Angel',
 'V.C. Andrews',
 'https://images.amazon.com/images/P/0671525433.01.MZZZZZZZ.jpg',
 'A Walk to Remember',
 'Nicholas Sparks',
 'https://images.amazon.com/images/P/0446608955.01.MZZZZZZZ.jpg',
 'GARDEN OF SHADOWS (Dollanger Saga (Paperback))',
 'Ariel Jennifer Jones',
 'https://images.amazon.com/images/P/067164257X.01.MZZZZZZZ.jpg',
 'The Wedding',
 'Nicholas Sparks',
 'https://images.amazon.com/images/P/0446532452.01.MZZZZZZZ.jpg']

Thats how i build a recommandation system , now lets make an app of it